In [ ]:
!wget "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt?raw=true"
import os
os.rename("toxic_original-c1212f89.ckpt?raw=true","toxic_original-c1212f89.ckpt")

In [ ]:
import torch
import transformers

MODEL_URLS = {
    "original": "./toxic_original-c1212f89.ckpt"
}

PRETRAINED_MODEL = None


def get_model_and_tokenizer(
    model_type, model_name, tokenizer_name, num_classes, state_dict
):
    model_class = getattr(transformers, model_name)
    model = model_class.from_pretrained(
        pretrained_model_name_or_path=None,
        config=model_type,
        num_labels=num_classes,
        state_dict=state_dict,
    )
    tokenizer = getattr(transformers, tokenizer_name).from_pretrained(model_type)
    
    return model, tokenizer


def load_checkpoint(model_type="original", checkpoint=None, device='cpu'):
    if checkpoint is None:
        checkpoint_path = MODEL_URLS[model_type]
        loaded = torch.hub.load_state_dict_from_url(checkpoint_path, map_location=device)
    else:
        loaded = torch.load(checkpoint)
        if "config" not in loaded or "state_dict" not in loaded:
            raise ValueError(
                "Checkpoint needs to contain the config it was trained \
                    with as well as the state dict"
            )
    class_names = loaded["config"]["dataset"]["args"]["classes"]
    # standardise class names between models
    change_names = {
        "toxic": "toxicity",
        "identity_hate": "identity_attack",
        "severe_toxic": "severe_toxicity",
    }
    class_names = [change_names.get(cl, cl) for cl in class_names]
    model, tokenizer = get_model_and_tokenizer(
        **loaded["config"]["arch"]["args"], state_dict=loaded["state_dict"]
    )

    return model, tokenizer, class_names


def load_model(model_type, checkpoint=None, device='cpu'):
    if checkpoint is None:
        model, _, _ = load_checkpoint(model_type=model_type, device=device)
    else:
        model, _, _ = load_checkpoint(checkpoint=checkpoint,device=device)
    return model


class Detoxify:
    def __init__(self, model_type="original", checkpoint=PRETRAINED_MODEL, device="cpu"):
        super(Detoxify, self).__init__()
        self.model, self.tokenizer, self.class_names = load_checkpoint(
            model_type=model_type, checkpoint=checkpoint, device=device
        )
        self.device = device
        self.model.to(self.device)


    @torch.no_grad()
    def predict(self, text):
        self.model.eval()
        inputs = self.tokenizer(
            text, return_tensors="pt", truncation=True, padding=True
        ).to(self.model.device)
        out = self.model(**inputs)[0]
        scores = torch.sigmoid(out).cpu().detach().numpy()
        results = {}
        for i, cla in enumerate(self.class_names):
            results[cla] = (
                scores[0][i]
                if isinstance(text, str)
                else [scores[ex_i][i].tolist() for ex_i in range(len(scores))]
            )
        return results

In [ ]:
model, tokenizer, classes = load_checkpoint(checkpoint = "./toxic_original-c1212f89.ckpt")

In [ ]:
tokenizer.save_pretrained("./tokenizer/")

In [ ]:
model.save_pretrained("./model/")

In [ ]:
classes